# Dependencies

In [ ]:
# # Main Functions:
# import cv2
# import numpy as np
# from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
# from tensorflow import keras
# from tensorflow.keras.utils import to_categorical
# from tensorflow.keras import layers
# from tensorflow.keras.callbacks import TensorBoard

# #  Functions:
# import os
# from glob import glob
# from matplotlib import pyplot as plt
# from collections import Counter

# # Developed Functions: 
# from Dev_Fun.Calibration_Show import Calib_Show, mediapipe_detection, draw_landmarks
# from Dev_Fun.Dataset_Capture import files_creation, dataset_capture
# from Dev_Fun.Pre_Process import pre_process, crop_to_person, sil_V1, sil_V2
# from Dev_Fun.Predict import pred_show, final_pred

# Calibration
<img src="https://www.researchgate.net/profile/Jungpil-Shin/publication/362351225/figure/fig3/AS:1183521695645696@1659184969422/Mediapipe-detects-33-nodes-of-the-human-pose.ppm" width="400px">

In [ ]:
#Feed Input Source
feed = 0
# Calib_Show(feed)

# Setup Folders

In [ ]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('Custom_Gait_Dataset_V.4')
EXPORTED = os.path.join('Exported')

# Persons that we try to detect
persons = np.array(['Lance Moheb','Seif Ibrahim','Mina Nady','Omar Amin','Mohmed Youssef','Hamza Tharwat'])
# persons = np.array(['Lance Moheb','Seif Ibrahim','Mina Nady','Omar Amin','Mohmed Youssef','Hamza Tharwat','Romario Nagy','Ziad Diaa'])

# 10 videos worth of data
no_sequences = 10

# Videos are going to be 30 frames in length
sequence_length = 90

persons_len = len(persons)
total_sequences = persons_len*no_sequences

# Capturing Dataset

In [ ]:
# files_creation(persons,no_sequences,DATA_PATH)
# dataset_capture(persons,no_sequences,DATA_PATH,sequence_length,feed)

# Pre-Processing

* A) Importing Dataset

In [ ]:
# label_map = {label:num for num, label in enumerate(persons)}
# print(label_map)

In [ ]:
# sequences, labels = [], []
# for person in persons:
#     for sequence in np.array(os.listdir(os.path.join(DATA_PATH, person))).astype(int):
#         window = []
#         for frame_num in range(sequence_length):

#             image_path = os.path.join(DATA_PATH, person, str(sequence), f"{frame_num}.jpg")
#             print (image_path)
#             res = cv2.imread(image_path)
#             window.append(res)
#         sequences.append(window)
#         labels.append(label_map[person])

In [ ]:
# seqq=np.array(sequences)
# print(seqq.shape)
# show_test = cv2.cvtColor(seqq[0][30], cv2.COLOR_BGR2RGB)
# plt.imshow(show_test)

* B) dataset preproccesing
    * Flip Horizontal
    * Converting to Silhouette
        * Background Subtraction
        * Noise Reduction
        * Binarization
    * convert to gray-scale
    * Resize to CASIA-B Resolution (320x240)

In [ ]:
# proccessed = []
# fgbg = cv2.createBackgroundSubtractorKNN(detectShadows=False)
# for sequence in  range(total_sequences):
# # for sequence in  range(1):
#     for frame_num in range(sequence_length):

#         frame = cv2.cvtColor(seqq[sequence][frame_num], cv2.COLOR_BGR2RGB)

#         flip = cv2.flip(frame, 1)

#         sil = sil_V2(flip)

#         gray = cv2.cvtColor(sil, cv2.COLOR_BGR2GRAY)

#         resize = cv2.resize(gray,(320,240))

#         crop = crop_to_person(resize)

#         proccessed.append(crop)
        
#         print (len(proccessed))

In [ ]:
# plt.imshow(proccessed[30],cmap='gray')

* C) Export the Custom dataset with CASIA-B File Structure

In [ ]:
# def create_directories_if_not_exist(path):
#     if not os.path.exists(path):
#         os.makedirs(path)

In [ ]:
# # Define your dataset path
# dataset_path = "CASIA-B"

# # Define walking conditions
# walking_conditions = [
#     "bg-01", "bg-02",
#     "cl-01", "cl-02",
#     "nm-01", "nm-02", "nm-03", "nm-04", "nm-05", "nm-06"
# ]

# # Iterate over subjects
# for subject_id in range(119, 128):
#     subject_folder = os.path.join(dataset_path, str(subject_id))

#     # Iterate over walking conditions
#     for walking_condition in walking_conditions:
#         walking_condition_folder = os.path.join(subject_folder, walking_condition,str("090"))

#         # Iterate over frames
#         for frame_num in range(90):
#             # Define frame name
#             frame_name = f"{subject_id}-{walking_condition}-090-{frame_num:03d}.png"
#             frame_path = os.path.join(walking_condition_folder, frame_name)

#             walking_condition_index = walking_conditions.index(walking_condition)
#             index = (subject_id - 119) * len(walking_conditions) * 90 + walking_conditions.index(walking_condition) * 90 + frame_num
#             print(index)

#             # Check if the image is empty
#             if proccessed[index] is None:
#                 # Create a white image
#                 image = np.ones((128, 64, 3), dtype=np.uint8) * 255  # Assuming 128x64 is the size of your image
#             else:
#                 image = proccessed[index]

#             create_directories_if_not_exist(walking_condition_folder)
#             cv2.imwrite(frame_path, image)
#             print(f"Exported: {frame_path}")

# print("Images exported successfully!")


* D) Pre-Proccesing CASIA-B

In [ ]:
def import_images(input):
  image_pattern = os.path.join(input, "*.png")
  image_paths = glob(image_pattern)

  return image_paths

In [ ]:
# # Define your dataset path
# dataset_path = "CASIA-B"
# export_path = "exported"

# # Define walking conditions
# walking_conditions = [
#     "bg-01", "bg-02",
#     "cl-01", "cl-02",
#     "nm-01", "nm-02", "nm-03", "nm-04", "nm-05", "nm-06"
# ]

# # Iterate over subjects
# for subject_id in range(1,119):
#     subject_folder = os.path.join(dataset_path, f"{subject_id:03d}")
#     subject_folder_export = os.path.join(export_path, f"{subject_id:03d}")

#     # Iterate over walking conditions
#     for walking_condition in walking_conditions:
#         walking_condition_folder = os.path.join(subject_folder, walking_condition,str("090"))
#         walking_condition_folder_export = os.path.join(subject_folder_export, walking_condition,str("090"))
#         create_directories_if_not_exist(walking_condition_folder_export)
        
#         image_list = import_images(walking_condition_folder)
#         for image_path in image_list:

#             img = cv2.imread(image_path)
#             gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#             crop = crop_to_person(gray)

#             new_path = image_path.replace("CASIA-B", "exported")

#             if crop is None:
#                 break
#             else:
#                 cv2.imwrite(new_path, crop)

#             print(new_path, "###DONE###")
# print("Images exported successfully!")


# Train

A) Splitting Dataset

In [ ]:
# subjects = np.array([f"{i:03d}" for i in range(1, 125)])
# # subjects = np.concatenate((numbers_array, persons))
# print(len(subjects))
# print(subjects)


In [ ]:
# label_map = {label:num for num, label in enumerate(subjects)}
# print(label_map)

In [ ]:
# # Define your dataset path
# dataset_path = "Custom_Gait_Dataset_V.5"
# sequneces_train, sequneces_test, labels_train, labels_test  = [], [], [], []

# # Define walking conditions
# walking_conditions_train = [
#     "nm-01", "nm-02", "nm-03", "nm-04"
# ]
# walking_conditions_test = [
#     "nm-05", "nm-06"
# ]

# error_num = 0
# error = []


# # ---Taining Set---
# for subject_id in subjects:
#     subject_folder = os.path.join(dataset_path, subject_id)

#     set = []
#     # Iterate over walking conditions
#     for walking_condition in walking_conditions_train:
#         walking_condition_folder = os.path.join(subject_folder, walking_condition,str("090"))
        
#         window = []

#         image_list = import_images(walking_condition_folder)
#         for image_path in image_list[:30]:
#             img = cv2.imread(image_path)
#             gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#             print(image_path)
#             window.append(gray)
#         print("------------------------------------------")
#         # set.append(window)
#         labels_train.append(label_map[str(subject_id)])
#         sequneces_train.append(window)

#         if(len(window)!= 30):
#             error_num += 1
#             error.append(subject_id)
#     print("#################################################################")
    

# # ---Testing Set---
# for subject_id in subjects:
#     subject_folder = os.path.join(dataset_path, subject_id)

#     set = []
#     # Iterate over walking conditions
#     for walking_condition in walking_conditions_test:
#         walking_condition_folder = os.path.join(subject_folder, walking_condition,str("090"))
        
#         window = []

#         image_list = import_images(walking_condition_folder)
#         for image_path in image_list[:30]:
#             img = cv2.imread(image_path)
#             gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#             print(image_path)
#             window.append(gray)
#         print("------------------------------------------")
#         # set.append(window)
#         labels_test.append(label_map[str(subject_id)])
#         sequneces_test.append(window)

#         if(len(window)!= 30):
#             error_num += 1
#             error.append(subject_id)
#     print("#################################################################")
    
# print("Images loaded successfully!")
# print("errors " + str(error_num))
# print("error list:  " + str(error))


In [ ]:
# x_train = np.array(sequneces_train)
# x_test = np.array(sequneces_test)
# y_train = to_categorical(labels_train).astype(int)
# y_test = to_categorical(labels_test).astype(int)

In [ ]:
# print(x_train.shape)
# print(x_test.shape)
# print(y_train.shape)
# print(y_test.shape)

B) Building CNN Architecture

In [ ]:
# log_dir = os.path.join('Logs')
# tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
# def cnn_model():
#   model = keras.Sequential([
#     # First convolutional block
#     layers.Conv2D(32, (3, 3), activation="relu", input_shape=(30, 64, 44)),
#     layers.MaxPooling2D((2, 2)),

#     # Second convolutional block
#     layers.Conv2D(64, (3, 3), activation="relu"),
#     layers.MaxPooling2D((2, 2)),

#     # Third convolutional block
#     layers.Conv2D(128, (3, 3), activation="relu"),
#     layers.MaxPooling2D((2, 2)),

#     # Flatten the output of the convolutional layers
#     layers.Flatten(),

#     # Fully connected layers
#     layers.Dense(128, activation="relu"),
#     layers.Dense(256, activation="relu"),

#     # Output layer with the number of classes (modify this based on your dataset)
#     layers.Dense(124, activation="softmax")
#   ])

#   # Compile the model
#   model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

#   return model

In [ ]:
# model = cnn_model()

In [ ]:
# model.fit(x_train, y_train, batch_size=32 , epochs=20, callbacks=[tb_callback])

C) Training Results

In [ ]:
# model.summary()

In [ ]:
# print("Evaluate on test data")
# results = model.evaluate(x_test, y_test)
# print("test loss, test acc:", results)

In [ ]:
# res = model.predict(x_test)
# index = 2
# tst = subjects[np.argmax(res[index])]
# trn = subjects[np.argmax(y_test[index])]
# print("Subject tested:" + tst + "\n" + "Subject trained:" + trn)

In [ ]:
# model.save('train1.h5')

In [ ]:
# yhat = model.predict(x_test)
# ytrue = np.argmax(y_test, axis=1).tolist()
# yhat = np.argmax(yhat, axis=1).tolist()

# print(f"\n ##########< Accuracy: {(accuracy_score(ytrue, yhat)*100):.2f}% >########## \n")

# print(multilabel_confusion_matrix(ytrue, yhat))


# Detection

In [1]:
from Dev_Fun.Predict import final_pred

Using cache found in C:\Users\lance/.cache\torch\hub\pytorch_vision_v0.6.0
c:\Users\lance\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\lance\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model has been loaded.


Using cache found in C:\Users\lance/.cache\torch\hub\pytorch_vision_v0.6.0


Model has been loaded.


In [2]:
name = final_pred("Test_Samples/test4.mp4","Saved_Models/train_55_6.h5")



not pushed
not pushed
not pushed
not pushed
1/1 [==============================] - 0s 115ms/step
Seif Ibrahim
1/1 [==============================] - 0s 20ms/step
Mina Nady
1/1 [==============================] - 0s 25ms/step
Mina Nady
1/1 [==============================] - 0s 20ms/step
Mina Nady
1/1 [==============================] - 0s 20ms/step
Mina Nady
1/1 [==============================] - 0s 20ms/step
Mina Nady
1/1 [==============================] - 0s 20ms/step
Mina Nady
1/1 [==============================] - 0s 20ms/step
Mina Nady
1/1 [==============================] - 0s 21ms/step
Mina Nady
1/1 [==============================] - 0s 19ms/step
Mina Nady
1/1 [==============================] - 0s 20ms/step
Mina Nady
End of video.
predictions array: ['Seif Ibrahim', 'Mina Nady', 'Mina Nady', 'Mina Nady', 'Mina Nady', 'Mina Nady', 'Mina Nady', 'Mina Nady', 'Mina Nady', 'Mina Nady', 'Mina Nady']
The most repeated element is: Mina Nady 
Percentage:  90.9090909090909
Mina Nady
Mina Na

In [3]:
print(name)

Mina Nady
